# Реализация посимвольной генерации текста на основе LSTM

В данном примере мы используем тексты из произведений Ницше, немецкого философа конца XIX века (в переводе на английский язык). Таким образом, в результате обучения у нас получится языковая модель, обладающая специфическими особенностями, характерными для произведений Ницше, а не обобщенная модель английского языка.

Загрузка и парсинг исходного текстового файла

In [ ]:
!pip install tensorflow keras numpy

In [ ]:
import keras
import numpy as np
path = keras.utils.get_file('nietzsche.txt', origin='https://github.com/minimaxir/textgenrnn/blob/master/datasets/reddit_apple_android_2000.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

      0/Unknown 0s 0s/stepCorpus length: 780271


Затем извлечем частично перекрывающиеся последовательности с длиной maxlen, выполним прямое кодирование и упакуем в трехмерный массив Numpy x с формой (последовательности, максимальная_длина, уникальные_символы). Одновременно подготовим массив y с соответствующими целями: векторы с символами, полученные прямым кодированием, которые следуют за каждой извлеченной последовательностью.
Векторизация последовательностей символов


In [ ]:
maxlen = 60 #Извлечение последовательностей по 60 символов
step = 3 #Новые последовательности выбираются через каждые 3 символа
sentences = [] #Хранение извлеченных последовательностей
next_chars = [] #Хранение целей (символов, следующих за последовательностями)
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text))) #Список уникальных символов в корпусе
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars) # Словарь, отображающий уникальные символы в их индексы в списке «chars»

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1 #Прямое кодирование символов в бинарные массивы

Number of sequences: 260071
Unique characters: 108
Vectorization...


Конструирование сети
Эта сеть состоит из единственного слоя LSTM, за которым следует классификатор Dense с функцией softmax выбора из всех возможных символов. Но имейте в виду, что рекуррентные нейронные сети не единственный способ генерирования после-довательностей данных; одномерные сверточные сети тоже показали превосходные результаты в решении этой задачи.
Модель с единственным слоем LSTM для предсказания следующего символа


In [ ]:
from keras import layers
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))


Так как цели имеют формат прямого кодирования, используем для обучения модели функцию потерь categorical_crossentropy.
Конфигурация компилируемой модели


In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Обучение модели и извлечение образцов из нее
Имея обученную модель и фрагмент начального текста, можно сгенерировать новый текст, выполнив следующие пункты:
1. Извлечь из модели распределение вероятностей следующего символа для имеющегося на данный момент сгенерированного текста.
2. Выполнить взвешивание распределения с заданной температурой.
3. Выбрать следующий символ в соответствии с вновь взвешенным распределением вероятностей.
4. Добавить новый символ в конец текста.
Вот код, который мы используем для взвешивания оригинального распределения вероятностей, возвращаемого моделью, и извлечения индекса символа (функция выборки).
Функция выборки следующего символа с учетом прогнозов модели


In [ ]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

Наконец, следующий цикл повторяет обучение и генерирует текст. Для начала сгенерируем текст, использовав разные температуры после каждой эпохи. Это позволит вам увидеть, как меняется генерируемый тест по мере схождения модели и как температура влияет на стратегию выбора.
Цикл генерации текста


In [ ]:
import random
import sys
import numpy as np

for epoch in range(1, 61):  # Обучение модели в течение 60 эпох
    print('epoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)

    # Генерация текста только на 15, 30 и 60 эпохах
    if epoch in [15, 30, 60]:
        # Выбор случайного начального текста
        start_index = random.randint(0, len(text) - maxlen - 1)
        generated_text = text[start_index: start_index + maxlen]
        print('\n--- Epoch {} - Generating with seed: "{}"'.format(epoch, generated_text))

        for temperature in [0.2, 0.5, 1.0, 1.2]:
            print('\n------ temperature:', temperature)
            sys.stdout.write(generated_text)

            # Генерация 400 символов
            for i in range(400):
                sampled = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(generated_text):
                    sampled[0, t, char_indices[char]] = 1.

                preds = model.predict(sampled, verbose=0)[0]
                next_index = sample(preds, temperature)
                next_char = chars[next_index]

                generated_text += next_char
                generated_text = generated_text[1:]
                sys.stdout.write(next_char)
                sys.stdout.flush()
            print('\n' + '-'*50)  # Разделитель между разными температурами
        print('\n' + '='*70 + '\n')  # Разделитель между разными эпохами

epoch 1
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 1.8846
epoch 2
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.0306
epoch 3
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.9432
epoch 4
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.9036
epoch 5
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.4755
epoch 6
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.3106
epoch 7
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 2.2747
epoch 8
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 2.3352
epoch 9
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.3960
epoch 10
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 3.1762
epoch 11
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 2.1127
epoch 12
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.4214
epoch 13
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.1367
epoch 14
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.0354
epoch 15
2032/2032 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - lo

<ipython-input-16-93e590fbe047>:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


 ver ta:tre-le" 1ine siaut="pie ne-cine-con="ere tbexts te,le-1.qbexl rpatine-dind gi> 0f fa-<divigte clay fle# taqupte n re-he gigsiton'te ht 1"><divinkig-./div><ddive-nes reatine-cobelosel>itig-veacte nd d visg deas me-numin rit-de="reacton:di bux 1evteing-.rdaost mine="_la-line=in cla-text" divl=s.umid><divginghin=t-_6pextinghext"_tertexnle vinest:/gine:che-fime:1.ckl fin
--------------------------------------------------

------ temperature: 0.5
n=t-_6pextinghext"_tertexnle vinest:/gine:che-fime:1.ckl fine-numb%  htex=" 25vt:&qule-minght:1di ginearelathingueil="comitg min>ges rig text="t:f;cov></div></diviv></viv><div 0    ff="6pv"me-sting-n"mrive-ces.="rime"><div><div deigst gont="de="liv></tita-linl>ss: react-cext="te re; midin="re-cha-dive" htoa-2iner ht 0vighit 6a claasiost="ple="nere-ce-ceact he soda>igs casd="mive-he{:1t mingt-cing_asto fid="mine-hi_sagi-di4e tatxn:t;ts: xth":/dionge-hehest=
--------------------------------------------------

------ temperature: 1.0
asto fid=

Мы выбрали случайный начальный текст «new faculty, and the jubilation reached its climax when kant»1. Далее показано, что получилось после 20-й эпохи, задолго до того, как модель полностью сошлась, для temperature=0.2:
new faculty, and the jubilation reached its climax when kant and such a man
in the same time the spirit of the surely and the such the such
as a man is the sunligh and subject the present to the superiority of the
special pain the most man and strange the subjection of the
special conscience the special and nature and such men the subjection of the
special men, the most surely the subjection of the special
intellect of the subjection of the same things and

Вот результат для temperature=0.5:
new faculty, and the jubilation reached its climax when kant in the eterned
and such man as it's also become himself the condition of the
experience of off the basis the superiory and the special morty of the
strength, in the langus, as which the same time life and "even who
discless the mankind, with a subject and fact all you have to be the stand
and lave no comes a troveration of the man and surely the
conscience the superiority, and when one must be w

А это результат для temperature=1.0:
new faculty, and the jubilation reached its climax when kant, as a
periliting of manner to all definites and transpects it it so
hicable and ont him artiar resull
too such as if ever the proping to makes as cnecience. to been juden,
all every could coldiciousnike hother aw passife, the plies like
which might thiod was account, indifferent germin, that everythery
certain destrution, intellect into the deteriorablen origin of moralian,
and a lessority o

После 60 эпох модель полностью сошлась и текст начал выглядеть более согласованным.
Вот результат для temperature=0.2:
cheerfulness, friendliness and kindness of a heart are the sense of the
spirit is a man with the sense of the sense of the world of the
self-end and self-concerning the subjection of the strengthorixes--the
subjection of the subjection of the subjection of the
self-concerning the feelings in the superiority in the subjection of the
subjection of the spirit isn't to be a man of the sense of the
subjection and said to the strength of the sense of the

Для temperature=0.5:
cheerfulness, friendliness and kindness of a heart are the part of the soul
who have been the art of the philosophers, and which the one
won't say, which is it the higher the and with religion of the frences.
the life of the spirit among the most continuess of the
strengther of the sense the conscience of men of precisely before enough
presumption, and can mankind, and something the conceptions, the
subjection of the sense and suffering and the

И для temperature=1.0:
cheerfulness, friendliness and kindness of a heart are spiritual by the
ciuture for the
entalled is, he astraged, or errors to our you idstood--and it needs,
to think by spars to whole the amvives of the newoatly, prefectly
raals! it was
name, for example but voludd atu-especity"--or rank onee, or even all
"solett increessic of the world and
implussional tragedy experience, transf, or insiderar,--must hast
if desires of the strubction is be stronges

Как видите, при низком значении температуры генерируется часто повторяющийся и предсказуемый текст, однако локальная структура очень реалистична: в частности, все слова (слово является локальным шаблоном символов) — это действительные английские слова. При высоком значении температуры генерируется более интересный текст, неожиданный и даже творческий; в нем иногда появляются совершенно новые слова, кажущиеся правдоподобными (например, eterned и troveration). При высокой температуре внутренняя структура начинает разрушаться, и большинство слов выглядят как полуслучайные последовательности символов. Без сомнения, 0,5 — самая интересная температура для генерации текста в данном конкретном решении. Всегда пробуйте несколько стратегий выбора! Разумный баланс между изученной структурой и случайностью — вот что делает сгенерированные данные интересными.
Обратите внимание на то, что, обучая модель большего размера дольше и на большем объеме данных, можно добиться генерации текста, который выглядит еще реалистичнее. Однако не думайте, что когда-нибудь вам удастся сгенерировать осмысленный текст, разве только по чистой случайности: вы всего лишь выбираете образцы данных из статистической модели, в которой символы следуют за символами. Язык — это канал общения, и существуют различия между сутью общения и статистической структурой сообщений, которыми кодируется общение. Чтобы осознать это различие, проведите мысленный эксперимент: представьте, что человеческий язык позволял бы сжимать информацию при общении почти так же, как это делают компьютеры с цифровой информацией. Язык не потерял бы своей осмысленности, но утратил статистическую структуру, что сделало бы невозможным обучение языковой модели, как мы только что это проделали.
Задание. Подобрать длинное художественное произведение на русском и на английском языке, сравнимое с исходным документом – трудами Ницше. Произвести все шаги из этой практики и получит сгенерированные последовательности на основе ваших документов.
!!Обратите внимание на представление текстовых данных в документе с трудами Ницше. Ваш документ должен быть представлен в таком же фориате.


# Мой вариант

In [1]:
!pip install tensorflow keras numpy

In [7]:
import numpy as np
import random
import sys
from tensorflow import keras
from tensorflow.keras import layers



In [2]:
import keras
import numpy as np

path = keras.utils.get_file('pride_and_prejudice.txt',
                          origin='https://www.gutenberg.org/files/1342/1342-0.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

752575/752575 ━━━━━━━━━━━━━━━━━━━━ 1s 2us/step
Corpus length: 728842


In [3]:
path = keras.utils.get_file('crime_and_punishment.txt',
                          origin='https://www.gutenberg.org/files/2554/2554-0.txt')
text = open(path).read().lower()
print('Длина корпуса:', len(text))

1159924/1159924 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step
Длина корпуса: 1135214


In [10]:
import numpy as np
import random
import sys
from tensorflow import keras
from tensorflow.keras import layers

# Загрузка данных
path = keras.utils.get_file(
    'crime_and_punishment.txt',
    origin='https://www.gutenberg.org/files/2554/2554-0.txt'
)
text = open(path).read().lower()
print('Длина корпуса:', len(text))

# Подготовка данных
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)

# Векторизация
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# Модель
model = keras.Sequential([
    layers.Input(shape=(maxlen, len(chars))),  # Явное указание входного слоя
    layers.LSTM(128),
    layers.Dense(len(chars), activation='softmax')
])

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

# Функция выборки
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Обучение и генерация
for epoch in range(1, 60):
    print('Эпоха', epoch)
    model.fit(x, y, batch_size=128, epochs=1, verbose=1)

    if epoch in [1, 15, 30, 60]:
      start_index = random.randint(0, len(text) - maxlen - 1)
      generated_text = text[start_index: start_index + maxlen]
      print('--- Генерация с началом: "' + generated_text + '"')

      for temperature in [0.2, 0.5, 1.0, 1.2]:
          print('------ температура:', temperature)
          sys.stdout.write(generated_text)

          for i in range(400):
              sampled = np.zeros((1, maxlen, len(chars)))
              for t, char in enumerate(generated_text):
                  sampled[0, t, char_indices[char]] = 1.

              preds = model.predict(sampled, verbose=0)[0]
              next_index = sample(preds, temperature)
              next_char = chars[next_index]

              generated_text += next_char
              generated_text = generated_text[1:]
              sys.stdout.write(next_char)
              sys.stdout.flush()
          print()

Длина корпуса: 1135214
Эпоха 1
2957/2957 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - loss: 2.1356
--- Генерация с началом: "f, i entreat you, what are you doing?” pulcheria alexandrovn"
------ температура: 0.2
f, i entreat you, what are you doing?” pulcheria alexandrovna are it was a little to seem in the street. it’s a deed to the street. i am for the street. she was a little to seem it was a could her for the street. he was some to the street.

“i could not in the street. it’s seem in the street. it’s not in the street.

“why while it’s not the stood of the fact of the street. the liddle at the street. it’s a could to the street. it’s not the street. i am a st
------ температура: 0.5
. it’s a could to the street. it’s not the street. i am a striently dotrant of the street, then the stood, but it’s a possed on the street, who wondenstanding at the round her come as the chill at the tark indeed not is not the secome of the possion to a toment
of the street. he would not a be one
surprised to t

In [ ]:
import numpy as np
import random
import sys
from tensorflow import keras
from tensorflow.keras import layers

path = keras.utils.get_file(
    'crime_and_punishment.txt',
    origin='https://www.gutenberg.org/files/2554/2554-0.txt'
)

text = open(path).read().lower()
print('Длина корпуса', len(text))



maxlen = 60
step = 3
sentence = []
next_char = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_char.append(text[i + maxlen])

chars = sorted(list(set(text)))
char_indices = dict((char, char.index(char)) for char in chars)

x = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences)), len(chars), dtype=np.bool)


for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next-chars[i]]] = 1

model = keras.Sequential([
    layers.Input(shape=(maxlen, len(chars))),
    layers.LSTM(128),
    layers.Dense(len(chars), activation='softmax')
])


optimizer = keras.optimizer.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature = 1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)


for epoch in range(1, 61):



# Обучение и генерация
for epoch in range(1, 60):
    print('Эпоха', epoch)
    model.fit(x, y, batch_size=128, epochs=1, verbose=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Генерация с началом: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ температура:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()